# Activity 3

## Name: Shiv Parekh


#### Q1) (a) (5 points) Execute a single query to return and print out the total assets owned by the bank ( i.e., a cumulative total of the assets at all branches)

In [ ]:
!pip install mysql-connector-python
from IPython.display import display, Markdown
import mysql.connector

def print_table(cursor):
     headers=cursor.column_names
     result=cursor.fetchall()
     display(Markdown(f'```sql\nResult statement:\n{cursor.statement};\nNumber of rows affected: {cursor.rowcount}\n```'))
     border=["----------------"]*len(headers)
     format_row="{:<22}"*(len(headers))
     print(format_row.format(*headers))
     print(format_row.format(*border))
     for rows in result:
         print(format_row.format(*rows))
     print("\n")


def execute_query(conn):
    try:
        cursor=conn.cursor()
        sql_command='Select FORMAT(sum(assets),2) as Total_Assets from branch'
        cursor.execute(sql_command)
        print_table(cursor)
    except mysql.connector.Error as error:
        print(error)
        conn.rollback()
    conn.close()

def main():
    host='bankdb.cfyv4ccl5kny.us-east-1.rds.amazonaws.com'
    user='admin'
    password=open("password.txt").read()
    conn=mysql.connector.connect(host=host,
                        user=user,
                        password=password,
                        database='bankdb')
    execute_query(conn)
main()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


```sql
Result statement:
Select FORMAT(sum(assets),2) as Total_Assets from branch;
Number of rows affected: 1
```

Total_Assets          
----------------      
24,600,479.98         




#### Q1) (b) (10 points) Create a trigger named maintain_branch_size on your branch table that ensures that a small branch does not become a large branch (i.e., that its total assets never gets to 3 million dollars). Whenever an update to the assets of a small branch is attempted that would cause it to become a large branch, your trigger should store this excess money in a temporary table (e.g. bailout_cache) to be later divided equally among the existing large branches, capping the small branch at 2,999,999.99 in total assets.

In [ ]:
from IPython.display import display, Markdown
import mysql.connector

def print_table(cursor):
     headers=cursor.column_names
     result=cursor.fetchall()
     display(Markdown(f'```sql\nResult statement:\n{cursor.statement};\nNumber of rows affected: {cursor.rowcount}\n```'))
     border=["----------------"]*len(headers)
     format_row="{:<22}"*(len(headers))
     print(format_row.format(*headers))
     print(format_row.format(*border))
     for rows in result:
         print(format_row.format(*rows))
     print("\n")

def execute_query(conn):
    try:
        cursor=conn.cursor()
        sql_command = [];
        sql_command.append('''create table if not exists bailout_cache
        (
        id int not null auto_increment,
        balance  numeric(12,2),
        primary key(id)
        );''')
        sql_command.append('''CREATE TRIGGER maintain_branch_size BEFORE UPDATE ON branch FOR EACH ROW
        BEGIN DECLARE cache_amount NUMERIC(12, 2) DEFAULT 0;
        IF NEW.assets >=3000000 and OLD.assets<3000000 THEN SET cache_amount = NEW.assets-2999999.99;
        INSERT INTO bailout_cache (balance) VALUES (cache_amount);
        Set New.assets=2999999.99; END IF; END;''')

        for sql in sql_command:
            cursor.execute(sql)
            print_table(cursor)
            conn.commit()
    except mysql.connector.Error as error:
        print(error)
        conn.rollback()
    conn.close()

def main():
    host='bankdb.cfyv4ccl5kny.us-east-1.rds.amazonaws.com'
    user='admin'
    password=open("password.txt").read()
    conn=mysql.connector.connect(host=host,
                        user=user,
                        password=password,
                        database='bankdb')
    execute_query(conn)
main()

```sql
Result statement:
create table if not exists bailout_cache      
        (
        id int not null auto_increment,
        balance  numeric(12,2),
        primary key(id)
        );;
Number of rows affected: 0
```

```sql
Result statement:
CREATE TRIGGER maintain_branch_size BEFORE UPDATE ON branch FOR EACH ROW 
        BEGIN DECLARE cache_amount NUMERIC(12, 2) DEFAULT 0;   
        IF NEW.assets >=3000000 and OLD.assets<3000000 THEN SET cache_amount = NEW.assets-2999999.99; 
        INSERT INTO bailout_cache (balance) VALUES (cache_amount);
        Set New.assets=2999999.99; END IF; END;;
Number of rows affected: 0
```

#### Q1) (c) (5 points) Execute a query that attempts to divide the bailout money among the small branches as a test of your trigger.

In [ ]:
from IPython.display import display, Markdown
import mysql.connector

def print_table(cursor):
     headers=cursor.column_names
     result=cursor.fetchall()
     display(Markdown(f'```sql\nResult statement:\n{cursor.statement};\nNumber of rows affected: {cursor.rowcount}\n```'))
     border=["----------------"]*len(headers)
     format_row="{:<22}"*(len(headers))
     print(format_row.format(*headers))
     print(format_row.format(*border))
     for rows in result:
         print(format_row.format(*rows))
     print("\n")

def execute_query(conn):
    try:
        cursor=conn.cursor()
        sql_command1='Select count(*) from branch where assets<3000000'
        cursor.execute(sql_command1)
        result=cursor.fetchall()
        sql_command2='Select branch_name,branch_city,FORMAT(assets,2) as assets from branch where assets<3000000'
        cursor.execute(sql_command2)
        print_table(cursor)
        sql_command3='UPDATE branch SET assets=assets+(10000000)/(%s) WHERE assets<3000000;'
        cursor.execute(sql_command3,result[0])
        print_table(cursor)
        conn.commit()
        sql_command4='Select branch_name,branch_city,FORMAT(assets,2) as assets from branch where assets<3000000'
        cursor.execute(sql_command4)
        print_table(cursor)
    except mysql.connector.Error as error:
        print(error)
        conn.rollback()
    conn.close()

def main():
    host='bankdb.cfyv4ccl5kny.us-east-1.rds.amazonaws.com'
    user='admin'
    password=open("password.txt").read()
    conn=mysql.connector.connect(host=host,
                        user=user,
                        password=password,
                        database='bankdb')
    execute_query(conn)
main()

```sql
Result statement:
Select branch_name,branch_city,FORMAT(assets,2) as assets from branch where assets<3000000;
Number of rows affected: 6
```

branch_name           branch_city           assets                
----------------      ----------------      ----------------      
Central               Rye                   789,863.33            
Downtown              Brooklyn              1,289,583.33          
Mianus                Horseneck             789,783.33            
Perryridge            Horseneck             2,089,583.33          
Pownal                Bennington            789,583.33            
Redwood               Palo Alto             2,489,583.33          




```sql
Result statement:
UPDATE branch SET assets=assets+(10000000)/(6) WHERE assets<3000000;;
Number of rows affected: 6
```

```sql
Result statement:
Select branch_name,branch_city,FORMAT(assets,2) as assets from branch where assets<3000000;
Number of rows affected: 6
```

branch_name           branch_city           assets                
----------------      ----------------      ----------------      
Central               Rye                   2,456,530.00          
Downtown              Brooklyn              2,956,250.00          
Mianus                Horseneck             2,456,450.00          
Perryridge            Horseneck             2,999,999.99          
Pownal                Bennington            2,456,250.00          
Redwood               Palo Alto             2,999,999.99          




#### Q1) (d) (3 points) Execute a query that divides any excess money from the bailout (money in your bailout_cache table) among the large branches

In [ ]:
from IPython.display import display, Markdown
import mysql.connector

def print_table(cursor):
     headers=cursor.column_names
     result=cursor.fetchall()
     display(Markdown(f'```sql\nResult statement:\n{cursor.statement};\nNumber of rows affected: {cursor.rowcount}\n```'))
     border=["----------------"]*len(headers)
     format_row="{:<22}"*(len(headers))
     print(format_row.format(*headers))
     print(format_row.format(*border))
     for rows in result:
         print(format_row.format(*rows))
     print("\n")

def execute_query(conn):
    try:
        cursor=conn.cursor()
        values=[]
        sql_command1='Select branch_name,branch_city,FORMAT(assets,2) as assets from branch where assets>=3000000'
        cursor.execute(sql_command1)
        print_table(cursor)
        sql_command2='Select sum(balance) from bailout_cache'
        cursor.execute(sql_command2)
        result=cursor.fetchall()
        values+=result[0]
        sql_command3='Select count(*) from branch where assets>=3000000'
        cursor.execute(sql_command3)
        result=cursor.fetchall()
        values+=result[0]
        sql_command4='UPDATE branch SET assets=assets+(%s)/%s WHERE assets>=3000000;'
        cursor.execute(sql_command4,values)
        print_table(cursor)
        conn.commit()
        sql_command5='Drop table bailout_cache'
        cursor.execute(sql_command5)
        print_table(cursor)
        conn.commit()
        sql_command6='Select branch_name,branch_city,FORMAT(assets,2) as assets from branch where assets>=3000000'
        cursor.execute(sql_command6)
        print_table(cursor)
    except mysql.connector.Error as error:
        print(error)
        conn.rollback()
    conn.close()

def main():
    host='bankdb.cfyv4ccl5kny.us-east-1.rds.amazonaws.com'
    user='admin'
    password=open("password.txt").read()
    conn=mysql.connector.connect(host=host,
                        user=user,
                        password=password,
                        database='bankdb')
    execute_query(conn)
main()

```sql
Result statement:
Select branch_name,branch_city,FORMAT(assets,2) as assets from branch where assets>=3000000;
Number of rows affected: 3
```

branch_name           branch_city           assets                
----------------      ----------------      ----------------      
Brighton              Brooklyn              6,125,000.00          
North Town            Rye                   3,237,500.00          
Round Hill            Horseneck             7,000,000.00          




```sql
Result statement:
UPDATE branch SET assets=assets+(1912500.02)/3 WHERE assets>=3000000;;
Number of rows affected: 3
```

```sql
Result statement:
Drop table bailout_cache;
Number of rows affected: 0
```

```sql
Result statement:
Select branch_name,branch_city,FORMAT(assets,2) as assets from branch where assets>=3000000;
Number of rows affected: 3
```

branch_name           branch_city           assets                
----------------      ----------------      ----------------      
Brighton              Brooklyn              6,762,500.01          
North Town            Rye                   3,875,000.01          
Round Hill            Horseneck             7,637,500.01          




#### Q1) (e) (2 points) Execute a single query to return and print out the total assets owned by the bank ( i.e., a cumulative total of the assets at all branches) after all changes above have been made.

In [ ]:
from IPython.display import display, Markdown
import mysql.connector

def print_table(cursor):
     headers=cursor.column_names
     result=cursor.fetchall()
     display(Markdown(f'```sql\nResult statement:\n{cursor.statement};\nNumber of rows affected: {cursor.rowcount}\n```'))
     border=["----------------"]*len(headers)
     format_row="{:<22}"*(len(headers))
     print(format_row.format(*headers))
     print(format_row.format(*border))
     for rows in result:
         print(format_row.format(*rows))
     print("\n")

def execute_query(conn):
    try:
        cursor=conn.cursor()
        sql_command='Select FORMAT(sum(assets),2) Total_Assets from branch'
        cursor.execute(sql_command)
        print_table(cursor)
    except mysql.connector.Error as error:
        print(error)
        conn.rollback()
    conn.close()

def main():
    host='bankdb.cfyv4ccl5kny.us-east-1.rds.amazonaws.com'
    user='admin'
    password=open("password.txt").read()
    conn=mysql.connector.connect(host=host,
                        user=user,
                        password=password,
                        database='bankdb')
    execute_query(conn)
main()

```sql
Result statement:
Select FORMAT(sum(assets),2) Total_Assets from branch;
Number of rows affected: 1
```

Total_Assets          
----------------      
34,600,480.01         


